In [10]:
from openai import OpenAI
from dateutil.relativedelta import relativedelta
import os
import json
from pydantic import BaseModel,Field
from langchain.llms import OpenAI
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI, init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
from tqdm import tqdm
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

In [11]:
file_path = '../data/GSM8K/test.jsonl'

with open(file_path, "r", encoding="utf-8") as f:
    dataset = [json.loads(line) for line in f]

In [12]:
load_dotenv()
model=init_chat_model('gpt-4.1-mini',model_provider='openai',temperature=0.2)

In [ ]:
class Step(BaseModel):
    explanation: str
    output: str
class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

In [14]:
messages=[
    SystemMessage(content="You are a math expert. For every question, respond ONLY by calling the 'MathReasoning' tool. You must explain the solution step-by-step using 'steps', and include the final numeric result in 'final_answer'.The result should be a number only, with no units."),
    HumanMessage("Tìm x. 2x-3=9")
]
model_with_tools = model.with_structured_output(MathReasoning)
ai_msg = model_with_tools.invoke(messages)

In [15]:
ai_msg

MathReasoning(steps=[Step(explanation='Bắt đầu với phương trình 2x - 3 = 9.', output='2x - 3 = 9'), Step(explanation='Cộng 3 vào cả hai vế để loại bỏ -3 ở vế trái.', output='2x = 9 + 3'), Step(explanation='Tính tổng ở vế phải.', output='2x = 12'), Step(explanation='Chia cả hai vế cho 2 để tìm x.', output='x = 12 / 2'), Step(explanation='Tính thương để tìm giá trị của x.', output='x = 6')], final_answer='6')

In [16]:
def extract_ground_truth(answer: str) -> str:
    match = re.search(r"####\s*([\d,]+)", answer)
    if match:
        # Loại bỏ dấu phẩy để đảm bảo kết quả là số chuẩn
        return match.group(1).replace(",", "").strip()
    return ""

def compare_answers(predicted: str, actual: str, eps: float = 1e-3) -> bool:
    try:
        pred = round(float(predicted.strip()))
        act = round(float(actual.strip()))
        return abs(pred - act) < eps
    except ValueError:
        return predicted.strip().lower() == actual.strip().lower()

In [17]:
def process_item(item):
    question = item["question"]
    true_answer = extract_ground_truth(item["answer"])
    try:
        messages = [
        SystemMessage(content="""
        You are a math expert.
        For every question, you **must** respond using the `MathReasoning` tool.
        - Do not respond with plain text or natural language.
        - Use a list of `Step`s to break down the reasoning.
        - Include a `final_answer` as a single number, no units or symbols.
        - If you cannot solve it, return a final_answer of "unknown".
        - When dealing with money, do not round to thousands unless explicitly stated.
        """),
        HumanMessage(content=question)
    ]
        ai_msg = model_with_tools.invoke(messages)
        predicted_answer = ai_msg.final_answer
        
        return {
            "question": question,
            "true_answer": true_answer,
            "step":ai_msg.steps,
            "predicted_answer": predicted_answer,
            "correct": compare_answers(predicted_answer, true_answer)
        }
    except Exception as e:
        return {"error": str(e), "question": question}

results = []
correct = 0
total = len(dataset[:300])
with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(process_item, item) for item in dataset[:300]]
    for future in tqdm(as_completed(futures), total=total):
        result = future.result()
        if "error" not in result:
            results.append(result)
            if result["correct"]:
                correct += 1
        else:
            print(f"Error on question: {result['question'][:60]}... => {result['error']}")
accuracy = correct / total * 100
print(f"Accuracy: {accuracy:.2f}% ({correct}/{total})")



100%|██████████| 300/300 [03:14<00:00,  1.55it/s]

Accuracy: 98.67% (296/300)


In [18]:
output_path = "CoT_results.json"
os.makedirs(os.path.dirname(output_path) or ".", exist_ok=True)
def custom_encoder(obj):
    if hasattr(obj, "model_dump"):
        return obj.model_dump()
    elif hasattr(obj, "dict"):
        return obj.dict()
    else:
        raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

wrong_answers = [r for r in results if not r.get("correct", False)]

# Ghi file JSON
try:
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(wrong_answers, f, ensure_ascii=False, indent=2, default=custom_encoder)
    print(f"Đã lưu {len(wrong_answers)} kết quả sai vào {output_path}")
except TypeError as e:
    print(f"Lỗi khi ghi file JSON: {e}")


Đã lưu 4 kết quả sai vào CoT_results.json
